In [1]:
from functools import reduce
from ff import base_repr, ff_for_prime, ff_for_primitive_polynomial, mult_poly

# BCH codes
### Motivation
We start with generalized Hamming codes - $(n = 2^m - 1, k = 2^m - m - 1)$ linear codes that can correct single errors. Steps:
 1. Take the parity-check matrix - its $n$ columns are all the non-zero elements of $V_m = GF(2)^m$ vector space in arbitrary order.
 2. We have $n$ possible locations of a single error, so we need syndromes with $m$ positions to identify each of them. If we want to identiy locations of two errors, then intuitively we'll need 2x as many parity-check rows - the former $m$ rows identify the one error, the latter $m$ rows identiy the other error.
 3. Since the 1st halves of parity-check matrix columns are unique, we can look at the second halves as values of a function $f: V_m \rightarrow V_m$ on the first halves.
 4. Now we just need to choose the function so that for any 2-position error vector we have a unique syndrome - then we can identify and correct the errors. But the syndromes are combinations of parity-check matrix columns! We arrive at a system of equations:
$$
\begin{align}
 u + v = s_1 \\
 f(u) + f(v) = s_2
\end{align}
$$
for all $u, v, s_1, s_2 \in GF(2^m)$ that is supposed to have at most one solution $u, v$. The most simple such function is $f(x) = x^3$.

It's parity-check matrix is
$$
H_2 = \begin{bmatrix}
 \alpha_0 && ... && \alpha_{n - 1} \\
 \alpha_0^3 && ... && \alpha_{n - 1}^3
\end{bmatrix}
$$
where $\alpha_0, ..., \alpha_{n-1}$ is any ordering of the $GF(2^m)$ field's non-zero elements. It has $2m$ rows as elements in $GF(2)$, so the dimension of the rowspace is $\leq 2m$, hence code dimension (length of the information word) is $\geq n - 2m$.

### Generalization
(theorem 9.1)

If we choose $n = 2^m$ - length of the codeword, and $t \leq \lfloor \frac{n - 1}{2} \rfloor$ - max number of errors to be corrected, then the $t \ x \ n$ matrix
$$
H_t = \begin{bmatrix}
 \alpha_0 && ... && \alpha_{n - 1} \\
 \alpha_0^3 && ... && \alpha_{n - 1}^3 \\
 && ... && \\
 \alpha_0^{2t - 1} && ... && \alpha_{n - 1}^{2t - 1}
\end{bmatrix}
$$

is the parity-check matrix of an $(n, k)$ code that can correct all error patterns of weight $\leq t$ and $k \geq n - tm$. These codes are called **BCH codes**.

### Cyclic BCH codes
BCH codes can be made cyclic by using $\alpha_0 = 1, \alpha_1 = \alpha ..., \alpha_{n - 1} = \alpha^{n-1}$, where $\alpha$ is the generator of non-zero elements of the $GF(2^m)$ field. Then we get nice encoders using shift registers for free. Decoding still needs more machinery to be developed since we've only seen burst error correction decoders so far and these codes are supposed to correct ANY error pattern of weight $\leq t$.

### Cyclic BCH code generator polynomial
From the form of $H_t$ it follows, that all the codewords (seen as polynomials $C_0 + ... + C_{n - 1} x^{n - 1}$ over $GF(2)$) satisfy $C(\alpha^i) = 0, \ i = 1, ..., 2t - 1$. The generator polynomial is the lowest degree monic polynomial that also satisfies this condition, so it's the minimal polynomial of the set $\{ \alpha, ..., \alpha^{2t - 1} \}$. Reasoning from the Appendix C shows that it will be a product of binomials with **conjugates** of these elements as roots. 

Phew, that's a mouthful - an example is needed!

### BCH generator polynomial example
Let's start by building a generator polynomial for BCH with given properties.

Following section 9.2 we know that for codeword length $n = 15$, correction of up to $t = 3$ errors the generator polynomial is going to be a product of minimal polynomials for $\alpha$, $\alpha^3$ and $\alpha^5$ (up to power of $5 = 2 t - 1$), where $\alpha$ is the primitive root of GF(16). Hence it's going to be the product of all binomials with roots being conjugate to these three, i.e. $\{\alpha, \alpha^2, \alpha^3, \alpha^4, \alpha^5, \alpha^6, \alpha^8, \alpha^9, \alpha^{10}, \alpha^{12}\}$.

According to the book the result is $g(x) = 1 + x + x^2 + x^4 + x^5 + x^8 + x^{10}$, let's see.

In [2]:
f_2 = ff_for_prime(2)
f_16_prim_pow = ff_for_primitive_polynomial(2, f_2, [1, 1, 0, 0, 1])

poly = [1]
for power in [1, 2, 3, 4, 5, 6, 8, 9, 10, 12]:
    # multiply by x - alpha^power
    poly = mult_poly(poly, [f_16_prim_pow.add_inv(f_16_prim_pow.power_to_num[power]), 1], f_16_prim_pow)

assert poly == [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]

Hell yeah, good to confirm again that field operations work fine!

So if we just implement finding conjugates for given powers of primitive element (which is simple), then we can automatically derive BCH generator polynomials for given code parameters $n, \ t$. At the moment, based on such polynomials we can at least implement encoding by generating appropriate generator matrix for cyclic code (or using shift registers).

### BCH generator polynomial automated derivation
Given a finite field $F_{p^m}$ and its element $a$, **conjugates** of $a$ are the unique elements of the sequence $a, a^p, ..., a^{p^{k - 1}}$ and their number $k$ is called the degree of $a$. $a$ belongs to the subfield $F_{p^k}$ but no smaller subfield.

In [3]:
# find conjugates of alpha in F_{p^m} where p is the characteristic of the field
def conjugates(alpha, p, field):
    result = [alpha]
    alpha_p = reduce(lambda x, y: field.mult(x, y), [alpha] * p)  # alpha^p
    alpha_pow = alpha_p

    while alpha_pow != alpha:
        result.append(alpha_pow)
        alpha_pow = field.pow(alpha_pow, p)
    
    return set(result)

# according to the example, the conjugates of alpha are [alpha, alpha^2, alpha^4, alpha^8] and conjugates of alpha^3 are [alpha^3, alpha^6, alpha^12, alpha^9]
expected = {f_16_prim_pow.power_to_num[p] for p in [1, 2, 4, 8]}
assert conjugates(f_16_prim_pow.power_to_num[1], 2, f_16_prim_pow) == expected

expected_3 = {f_16_prim_pow.power_to_num[p] for p in [3, 6, 12, 9]}
assert conjugates(f_16_prim_pow.power_to_num[3], 2, f_16_prim_pow) == expected_3

In [6]:
# p: the field characteristic
# field: GF(p^m)
# alpha: field primitive root
# t: max number of errors our code is able to correct
def bch_generator_polynomial(p, m, field, alpha, t):
    assert t <= (p ** m - 1) / 2

    # generate polynomial roots alpha, ..., alpha^{2t - 1]
    alpha_2 = field.pow(alpha, 2)
    next_root = alpha
    base_roots = []
    
    for _ in range(1, 2 * t + 1, 2):
        base_roots.append(next_root)
        next_root = field.mult(next_root, alpha_2)

    # find conjugates of the roots
    conjugate_sets = [conjugates(root, p, field) for root in base_roots]
    conjugates_final_set = reduce(lambda x, y: x | y, conjugate_sets)

    gen_poly = [1]
    for c in conjugates_final_set:
        gen_poly = mult_poly(gen_poly, [field.add_inv(c), 1], field)  # multiply all (x - conjugate) binomials

    return gen_poly

In [9]:
assert bch_generator_polynomial(p=2, m=4, field=f_16_prim_pow, alpha=f_16_prim_pow.power_to_num[1], t=3) == [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1]

Hell yeah! So we can automatically generate generator polynomials for BCH codes of selected properties!

# TODO:
 - implement BCH encoder generation
 - read how to decode these bastards (with error correction of course!) + implement decoder derivation (time & frequency domain, WTF)
 - move this shit elsewhere - this are not RDS fundamentals anymore!